In [7]:
import datautil

In [8]:
libraries = util.select_libraries_from_libraries_io()
migrations = util.select_migrations()
rules = util.select_rules(set(libraries["name"]))

print(len(migrations))
print(f"{len(set(migrations.fromLib))} from libs, {len(set(migrations.toLib))} to libs, {len(set(migrations.fromLib) | set(migrations.toLib))} total libs")
print(f"{len(set(migrations.startCommit) | set(migrations.endCommit))} total commits")
print(f"{len(set(migrations.repoName))} repositories")
print(f"{len(migrations[migrations.startCommit == migrations.endCommit])} startCommit == endCommit")
print(f"{len(set(zip(migrations.fromLib, migrations.toLib)))} migration rules")

13704
407 from libs, 621 to libs, 788 total libs
3215 total commits
1616 repositories
13281 startCommit == endCommit
1257 migration rules
